In [3]:
import olefile
import zlib
import struct

In [33]:
f = olefile.OleFileIO('논문요약.hwp')

In [40]:
f.listdir()

[['\x05HwpSummaryInformation'],
 ['BodyText', 'Section0'],
 ['DocInfo'],
 ['DocOptions', '_LinkDoc'],
 ['FileHeader'],
 ['PrvImage'],
 ['PrvText'],
 ['Scripts', 'DefaultJScript'],
 ['Scripts', 'JScriptVersion']]

In [58]:
enc = f.openstream('BodyText/Section0').read()
enc

b'\xacW;l\x1cU\x14\xbd\xb3;v\xd6\xc4&1\xc1\xf9(D\x0cR\x14ld\x19R \x10\xe2\xb3\x8e\x03v\x82\x91"cY Qd\xbc\xb3k\x0f\xde\x1f\xfb\xf3G\x14F\xa2\x00\x89"\x85EE\x91"\nA"\x05\x9d\x89\\8Q\x82M\x81\x04FD\x88j\xcb\x14\x14)A\n\x82\xf3\xee}ogf=\xb1\xb1\x94Y\xdd\xf1\xcc\x9b\xf7\xee=\xf7\xdc\xcf{>C\xcb\xd6\x8bDdS\x82\x9e"J&\xf0l\xd1\xd6J\xd7\r\xa2\x11\xfbrg\x82\xbcL\xb6J\xfaJ\xe0\xe7\xe5K\x99\xe0\xfd\x1d\x1aa\xf9\xf8{\x87j\x1b\xdf\xfc\xf0\xc1o\x0e97\xfb\xee\xda\x9b\x83[\xd7n\xf4\xdd}o\xc3\xa1\xcbk\xb5\x8d\xf5\x1f\xc7\xee8\xb4\xf2\xd3\xc5U\x87\xbe]O\xaf\xf6\x92\xe3\x14sFO/\xf5\xd0Y\x1b\x86q\xed\x83\x0cBR\x907\xedt\xc2\xccIw\x8a\\\xe8 jZ2v\xf4\xaa\xbaw\xd2\xa8}?ap\x16m\x9e\xfd4\x91E;\\\xe7R\xcb\x89\xcf\xee\x11\xfd1f\xd1\xcb\xcf\x10\xcb\xc9\xc3D_\x1d\x0c\xc4\\\xe7S\xeb-]\x03P\xfb/\xae\x89\xe4K\x89\xe3\x96e\xc5}\xff\xeb\xf9&\xa9\xef\x80\xc0\xe3o\xa7\x9at\xf3\xef\x85\xae\xcf;\xe4g\xf1\xc8\'\xfb\x8f?\x1b\x1dQ\x93\x83\x91Q\x10\xa2\xc8\xb6CHF\xed\xb4\xa5\x88\xb3\xb4-uM%\xc6\x97\xc6Rd%\xda<<\x93Z\xb6\x8e\xe8

In [64]:
dec = enc[1:41].decode('UTF-16')
dec

'㭗ᱬᑕ뎽瘻쓖ㄦ療䐨刌气ᥤ⁒\ue210躳瘃醂挢⁙摑'

In [29]:
import olefile
import zlib
import struct

def get_hwp_text(filename):
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()

    # HWP 파일 검증
    if ["FileHeader"] not in dirs or \
       ["\x05HwpSummaryInformation"] not in dirs:
        raise Exception("Not Valid HWP.")

    # 문서 포맷 압축 여부 확인
    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    # Body Sections 불러오기
    nums = []
    for d in dirs:
        if d[0] == "BodyText":
            nums.append(int(d[1][len("Section"):]))
    sections = ["BodyText/Section"+str(x) for x in sorted(nums)]

    # 전체 text 추출
    text = ""
    for section in sections:
        bodytext = f.openstream(section)
        data = bodytext.read()
        if is_compressed:
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data
    
        # 각 Section 내 text 추출    
        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in [67]:
                rec_data = unpacked_data[i+4:i+4+rec_len]
                section_text += rec_data.decode('utf-16')
                section_text += "\n"

            i += 4 + rec_len

        text += section_text
        text += "\n"

    return text

In [32]:
print(get_hwp_text('(양식-심사위원용) 개발보고서 (1).hwp'))

捤獥    汤捯    湰灧    
氠瑢    
2024 채용연계형 SW전문인재양성
개 발 보 고 서
2024. 8.
氠瑢    
프로젝트명
WH:ILE
팀원명
(팀장)
(팀원) 박건, 김현우, 오창선
氠瑢    
요 약 본
氠瑢    
프로젝트 정보
프로젝트명
WH:ILE
주제 영역
■ 생활   □ 업무   ■ 공공/교통   □ 금융/핀테크   □ 의료   □ 교육
□ 유통/쇼핑   ■ 엔터테인먼트
기술 분야
□ IoT
□ 가상현실
□ 모바일
■ 빅데이터
□ 데스트톱 SW
□ 자동제어기술
□ 인공지능
□ 블록체인
□ 보안
□ 영상처리
프로젝트 소개
- 추천 알고리즘을 이용한 관심사 취미 챌린지 SNS 플랫폼
- 허깅페이스와 코사인 유사도 분석을 통한 알고리즘 구현
개발 배경
및 필요성
1. 점진적인 목표 의식 저하
2. SNS 이용자 수 증가
3. 자기개발 관심도 및 자기개발 어플에 대한 수요 증가
4. 1인 가구 증가에 따른 사회적 고립감 극복
프로젝트 특·장점
선행 조건 없이 자유롭게 참여할 수 있는 챌린지 SNS 서비스
본질을 해치지 않는 직관적인 UI,UX
챌린지 달성을 목표로 관심사와 취미를 공유할 수 있는 SNS
주요 기능
추천 알고리즘을 활용한 인기, 관심 챌린지
챌린지 참여 및 신규 챌린지 등록
챌린지, 피드, 팔로워 검색
게시판, 댓글, 좋아요 등 관심도 표현 기능
마이페이지, 완료한 챌린지 조회
기대효과 및
활용 분야
효과적인 성취감 충족
사회적 고립감 극복 및 유대감 형성
교육기관 연계, 교육활동 지원 앱 개발
지역 명소, 특산물 등 챌린지와 관련된 지자체 프로모션
챌린지에 사용하거나 이를 보조하는 물품 공동구매
氠瑢    
본    문 
I. 프로젝트 개요 # 프로젝트개요는 2장 이내로 작성
 1. 프로젝트 소개
   1) 자기개발을 돕는 챌린지 SNS
    - 꾸준히 늘어나는 자기개발에 대한 관심도, 그러나 시간이 지남에 따라
      흐려지는 목표 의식
漠杳    
<새해 계획 수립 비율 및 달성여부

In [9]:
import olefile
import zlib
import struct
#### 추가 ####
import re
import unicodedata

class HWPExtractor(object):
    FILE_HEADER_SECTION = "FileHeader"
    HWP_SUMMARY_SECTION = "\x05HwpSummaryInformation"
    SECTION_NAME_LENGTH = len("Section")
    BODYTEXT_SECTION = "BodyText"
    HWP_TEXT_TAGS = [67]

    def __init__(self, filename):
        self._ole = self.load(filename)
        self._dirs = self._ole.listdir()

        self._valid = self.is_valid(self._dirs)
        if (self._valid == False):
            raise Exception("Not Valid HwpFile")
        
        self._compressed = self.is_compressed(self._ole)
        self.text = self._get_text()
	
    # 파일 불러오기 
    def load(self, filename):
        return olefile.OleFileIO(filename)
	
    # hwp 파일인지 확인 header가 없으면 hwp가 아닌 것으로 판단하여 진행 안함
    def is_valid(self, dirs):
        if [self.FILE_HEADER_SECTION] not in dirs:
            return False

        return [self.HWP_SUMMARY_SECTION] in dirs

	# 문서 포맷 압축 여부를 확인
    def is_compressed(self, ole):
        header = self._ole.openstream("FileHeader")
        header_data = header.read()
        return (header_data[36] & 1) == 1

	# bodytext의 section들 목록을 저장
    def get_body_sections(self, dirs):
        m = []
        for d in dirs:
            if d[0] == self.BODYTEXT_SECTION:
                m.append(int(d[1][self.SECTION_NAME_LENGTH:]))

        return ["BodyText/Section"+str(x) for x in sorted(m)]
	
    # text를 뽑아내는 함수
    def get_text(self):
        return self.text

	# 전체 text 추출
    def _get_text(self):
        sections = self.get_body_sections(self._dirs)
        text = ""
        for section in sections:
            text += self.get_text_from_section(section)
            text += "\n"

        self.text = text
        return self.text

	# section 내 text 추출
    def get_text_from_section(self, section):
        bodytext = self._ole.openstream(section)
        data = bodytext.read()

        unpacked_data = zlib.decompress(data, -15) if self.is_compressed else data
        size = len(unpacked_data)

        i = 0

        text = ""
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            level = (header >> 10) & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in self.HWP_TEXT_TAGS:
                rec_data = unpacked_data[i+4:i+4+rec_len]
                
                ############## 정제 추가된 부분 #############
                decode_text = rec_data.decode('utf-16')
                # 문자열을 담기 전 정제하기
                res = remove_control_characters(remove_chinese_characters(decode_text))
                
                text += res
                text += "\n"

            i += 4 + rec_len

        return text

#################### 텍스트 정제 함수 #######################
    # 중국어 제거
def remove_chinese_characters(s: str):   
    return re.sub(r'[\u4e00-\u9fff]+', '', s)
    
# 바이트 문자열 제거
def remove_control_characters(s):    
    return "".join(ch for ch in s if unicodedata.category(ch)[0]!="C")
        
        
# text 추출 함수 -> 이 함수를 사용하면 됨
def hwp_get_text(filename):
    hwp = HWPExtractor(filename) 
    return hwp.get_text()
# 출처: https://code-angie.tistory.com/49 [CodeAngie:티스토리]

In [11]:
print(hwp_get_text('논문요약.hwp'))

MCMC를 이용한 선형전투모형의 베이지안 추론 사례†
1) (주)심네트 R&D 및 V&V연구소
교신저자 : bjyoo@korea.ac.kr
(Bayesian Inference for the Linear Combat Model by using MCMC)
유병주1†
(Byung Joo Yoo)
ABSTRACT
  As a way of predicting the future combat by analyzing past combat data, a method of fitting and analyzing the linear combat model may be considered. If two kinds of data differ historically from each other, it is difficult to fit into a single linear combat model due to violation of model assumptions. Two data of the same time can be applied to one model such as by using the dummy variable. but in this case, it is difficult to construct one model because it is two heterogeneous data with different ages. To solve this problem, Bayesian inference method using MCMC is proposed. After deriving the posterior distribution based on the noninformative prior and the previous period data, it is used as an informative prior for obtaining the final posterior distribution based on the next period data, then can predict the futu